# Agent Development
For making the actual itinerary reccomendations, we want to build a conversational agent, which handles querying the formatting a query to the graph database, selecting the most appropriate events based on the user's query and the reccomendation scores, and then writing a helpful output message explaining the logic they used to reach their conclusion. To do this, we will need an agent that requires three iterations per user query:
1. **Iteration 1**: Format a Graph DB query based on the user's input. The results of this query should return the top 10 locations of a certain type in a given city.
2. **Iteration 2**: Anaylze the results of the graph query, select the 3-5 most appropriate locations/activites to suggest to the user. Format these activities as events, with a `title`, `url`, `start_date`, and `end_date`.
3. **Iteration 3**: Review the results of the event generation, format a friendly message to the user explaining the agent's choices and asking for any additional information that might be needed.

As a general fallback, the agent should not reccomend any activites if the user's query cannot be used to guide a query on the venue graph. In this event, the agent should provide a response in the first iteration, explaining what the user needs to change about their query to provide more information.


## Tool Creation

The first thing that we need to take care of to setup the agent is to create the tool objects that the agent will have access too. This will require writing python functions as well as OpenAI tool definition objects, which should be mapped to the corresponding python function. We will want to create the following tools:
1. `venue_query(city: CityName, activity: ActivityType) -> List[VenueResult]`
2. `event_formatter(venues: List[VenueResult], start_time: str, end_time: str) -> List[EventObject]`

We will first need to create the custom objects that will be used by these functions, then implement the functions themselves, then finally, create the tool definitions, adhering to the OpenAI tool schema.

In [67]:
import urllib.parse
from enum import Enum
from datetime import datetime
from typing import Dict, Any
from pydantic import BaseModel

TIME_FORMAT = r"%Y-%m-%dT%H:%M:%S"
TIME_FORMAT_ALT = r"%Y-%m-%d %H:%M:%SZ"
PTG_USER_ID = "user_2aMmpSqdJphUB3IRp5lXWhH3Edw"

class CityName(Enum):
    NEW_YORK = 'NYC'
    LOS_ANGELES = 'LA'
    MIAMI = 'MIAMI'
    CHICAHO = 'CHICAGO'
    SCOTTSDALE = 'SCOTTSDALE'

    @staticmethod
    def schema():
        """Return the JSON schema of this object as a stringified JSON object."""
        return {
            "type": "string",
            "enum": [
                "NYC",
                "LA",
                "MIAMI",
                "CHICAGO",
                "SCOTTSDALE"
            ]
        }

class ActivityType(Enum):
    RESTAURANT = 'restaurant'
    ACTIVITY = 'activity'
    ENTERTAINMENT = 'entertainment'

    @staticmethod
    def schema():
        """Return the JSON schema of this object as a stringified JSON object."""
        return {
            "type": "string",
            "enum": [
                "restaurant",
                "activity",
                "entertainment"
            ]
        }

class VenueInformation(BaseModel):
    """The information about a venue."""
    name: str
    description: str

class VenueResult(VenueInformation):
    """VenueResult object that represents a venue result from the API."""

    relevance_score: float

class Event(BaseModel):
    """Event object that represents an event at a venue."""
    title: str
    start_time: str
    end_time: str

    def __init__(self, **kwargs):
        # We want to assert that the sart_time is before the end_time

        super().__init__(**kwargs)

        start, end = self.times
        assert start < end, "start_time must be before end_time."

    def __str__(self) -> str:
        start, end = self.times
        return f"\t{self.title}:\n\t{'-' * (len(self.title) + 1)}\n\tStart: {start}\n\tEnd: {end}\n"    

    @property
    def times(self):
        try:
            start = datetime.strptime(self.start_time, TIME_FORMAT)
            end = datetime.strptime(self.end_time, TIME_FORMAT)
        except ValueError:
            start = datetime.strptime(self.start_time, TIME_FORMAT_ALT)
            end = datetime.strptime(self.end_time, TIME_FORMAT_ALT)
        return start, end

    @classmethod
    def from_venue(cls, venue: VenueInformation, start_time: str = None, end_time: str = None) -> 'Event':
        """Create an Event object from a VenueResult object."""
        # Validate the input arguments.
        assert type(venue) == VenueInformation, "venue must be a VenueInformation object."
        return cls(title=venue.name, start_time=start_time, end_time=end_time)


In [60]:
import os
from typing import List

from neo4j import GraphDatabase
from neo4j.graph import Node

DB_USER = os.getenv("NEO4J_DATABASE_USERNAME")
DB_URL = os.getenv("NEO4J_DATABASE_URL")
DB_PASSWORD = os.getenv("NEO4J_DATABASE_PASSWORD")


def graph_driver() -> List[Node]:
    """Execute a query on the Neo4j database."""
    driver = GraphDatabase.driver(DB_URL, auth=(DB_USER, DB_PASSWORD))
    return driver


In [61]:
import pinecone

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")
PINECONE_INDEX = os.getenv("PINECONE_INDEX")

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
index = pinecone.Index(PINECONE_INDEX)

In [62]:
import random
import tiktoken
import time

from openai import OpenAI
from pinecone.core.client.model.scored_vector import ScoredVector

class VenueQueryTool(BaseModel):

    category: ActivityType
    city: CityName
    query: str


    def __init__(self, debug: bool = False, **kwargs):
        super().__init__(**kwargs)
        self._openai_client = OpenAI()
        self._debug = debug
        

    def _embed_query(self, query: str) -> List[float]:
        """Embed a query using the OpenAI API."""
        # Get the embeddings of the query.
        response = self._openai_client.embeddings.create(
            input=query,
            model="text-embedding-ada-002"
        )
        return response.data[0].embedding

    def _query_vectorstore(self) -> List[ScoredVector]:
        """Query the Vectorstore database."""
        # Get a list of relevant venues from the Vectorstore database.
        query_value = self._embed_query(self.query)
        venue_results = index.query(
            query_value, 
            top_k=10, 
            namespace='venues', 
            include_metadata=True,
            filter = {
                'city': {'$eq': self.city.value},
                'category': {'$eq': self.category.value}
            }
        )

        assert 'matches' in venue_results, "'matches' field not found in response object"
        assert len(venue_results['matches']) > 0, "No venues found for the given query."

        return venue_results['matches']

    def __call__(self) -> List[VenueResult]:
        """Execute the VenueQueryTool."""
        if self._debug:
            start = time.perf_counter()
            print(f"\tQuerying for {self.category.value} in {self.city.value} with query: {self.query}")

        # Get a list of relevant venues from the Vectorstore database.
        filtered_venues = self._query_vectorstore()

        # Now that we have base venues, we need to order them based on the relationship values
        # between the venues and the user's posts

        # FILL IN GRAPH QUERY HERE

        # Format the results as VenueResult objects
        results = [VenueResult(**venue.metadata, relevance_score=venue.score) for venue in filtered_venues]

        if self._debug:
            end = time.perf_counter()
            print(f"\tFound {len(results)} results in {round(end - start, 2)} seconds.")
        return results

    @staticmethod
    def definition():
        return {
            "type": "function",
            "function": {
                "name": "venue_query",
                "description": "Detailed sentence describing the activity or location the user is looking for.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "category": ActivityType.schema(),
                        "city": CityName.schema(),
                        "query": {
                            "type": "string",
                            "description": "A one sentence description of the venue the user is looking for."
                        }
                    },
                    "required": ["category", "city", "query"]
                },
            }
        }

In [63]:
class EventCreatorTool(BaseModel):

    venues: List[VenueInformation]
    start_time: str
    end_time: str

    def __init__(self, debug: bool = False, **kwargs):
        super().__init__(**kwargs)
        self._debug = debug

    def __call__(self) -> List[Event]:
        """Execute the EventCreatorTool."""
        if self._debug:
            start = time.perf_counter()
            print(f"\tCreating {len(self.venues)} events from venues.")
        # We simply create an event for each venue.

        events = [Event.from_venue(venue, self.start_time, self.end_time) for venue in self.venues]

        if self._debug:
            end = time.perf_counter()
            print(f"\tCreated {len(events)} events in {round(end - start, 2)} seconds.")
        return events
        
    @staticmethod
    def definition():
        return {
            "type": "function",
            "function": {
                "name": "event_creator",
                "description": "Create events from a list of venues.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "venues": {
                            "type": "array",
                            "items": VenueInformation.model_json_schema()
                        },
                        "start_time": {
                            "type": "string",
                            "format": "%Y-%m-%dT%H:%M:%S"
                        },
                        "end_time": {
                            "type": "string",
                            "format": "%Y-%m-%dT%H:%M:%S"
                        }
                    },
                    "required": ["venues", "start_time", "end_time"]
                }
            }
        }


def execute_tool(_id: str, name: str, debug: bool = False, **kwargs):
    """Execute a tool given a name and a set of arguments."""
    if name == "venue_query":
        tool = VenueQueryTool(debug=debug, **kwargs)
    elif name == "event_creator":
        tool =  EventCreatorTool(debug=debug, **kwargs)
    else:
        raise ValueError(f"Invalid tool name: {name}.")
    result = tool()
    data = [item.model_dump() for item in result]
    return {
        "tool_call_id": _id,
        "role": "tool",
        "name": name,
        "content": json.dumps(data)
    }

tools = [VenueQueryTool.definition(), EventCreatorTool.definition()]

In [66]:
import json
from typing import Any, Tuple, Union

from openai import OpenAI

SYSTEM_PROMPT = """
You are a helpful travel agent. You help you're customers build their dream vacation. You are conversing
with a customer about an upcoming trip, helping them add activities to their itinerary.

You are provided with a list of the events that the customer has already added to their itinerary.

Your job is to converse with the customer to understand what activity or venue they are interested in 
adding to their itinerary. You will need to converse with the customer until you can write a detailed
1-2 sentence description of the exact activity or venue they are looking for.

Once you are confident that you have a detailed understanding of the activity the customer is looking for,
you can use the 'venue_query' tool to find a list of venues that match the customer's request. Once the 
results from the `venue_query` tool are returned, you must select the most 3-5 relevant results, and 
use the 'event_creator' tool to create a list of events for the customer to choose from.

YOU MUST CREATE AT LEAST 3 EVENTS FOR THE CUSTOMER TO CHOOSE FROM USING THE `event_creator` TOOL.

When you have a list of events to provide to the customer, you should write a brief statement summarizing
the events you have chosen. This summary should NOT be a list of the events, but rather a breif description
of the events you have chosen and why you chose them.
"""

ITINERARY_CONTEXT = """
City: {city}
Start Date: {start_date}
End Date: {end_date}
------------------------

Events:
------------------------
{events}
"""

class Agent:

    def __init__(self, user_id: str, model: str = 'gpt-3.5-turbo-1106', debug: bool = False):
        """Setup the agent object."""
        # The ID of the user that the agent is interacting with. This will be used for querying
        # the Graph Database.
        self._user_id = user_id

        self._tools = tools
        self.debug = debug

        self.messages = [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': self.itinerary}
        ]
        self.model = model
        self.client = OpenAI()
        self._finish_reason = None

    @property
    def itinerary(self):
        """Return the itinerary of the user. This must be fetched from the Graph Database."""
        driver = graph_driver()
        with driver.session() as session:
            result = session.run(
                """
                MATCH (i:Itinerary)-[:HAS_EVENT]->(e:Event) WHERE i.userId = $user_id
                RETURN i, collect(e) as events;
                """,
                user_id=self._user_id
            )
            # Fetch the records from the result
            record = result.single()
            assert record is not None, "User does not have an itinerary."
            itinerary = dict(record['i'].items())
            events = []
            for event in record['events']:
                data = dict(event.items())
                events.append(Event(title=data['title'], start_time=data['startTime'], end_time=data['endTime']))

        # Close the driver            
        driver.close()

        event_str = "\n".join([str(event) for event in events])
        itinerary_str = ITINERARY_CONTEXT.format(
            city=itinerary['city'],
            start_date=itinerary['startDate'],
            end_date=itinerary['endDate'],
            events=event_str
        )
        return itinerary_str


    @property
    def last_events(self) -> Union[List[Event], None]:
        """Return the last events."""
        # iterate backwards through self.messages until we find a message of role "tool",
        # with name "event_creator"
        for message in reversed(self.messages):
            if type(message) == dict and message['role'] == 'tool' and message['name'] == 'event_creator':
                data = json.loads(message['content'])
                events = [Event(**item) for item in data]
                return events
        return None

    def __call__(self, query: str) -> Tuple[str, List[Event]]:
        """Execute the agent."""
        self.messages.append({'role': 'user', 'content': query})

        count = 0
        while count < 5:
            # Execute a single step of the agent.
            self._step()

            # Get the last message from the agent (the result of the last step)
            last_message = self.messages[-1]

            # If the last message is a tool_call, execute the tool call.
            if self._finish_reason == 'tool_calls' and len(last_message.tool_calls) > 0:
                tool_calls = last_message.tool_calls

                # If the agent is producing multiple calls per step, that is a problem.
                assert len(tool_calls) == 1, "Only one tool call can be made at a time."
                tool_call = tool_calls[0]
                self._execute_tool(tool_call)

            # Otherwise, if the last message has content, then we may return the results.
            if self._finish_reason == 'stop':
                break
            
            # Increment the count to track iteratins
            count += 1
        
        return self.messages[-1].content, self.last_events

        
    def _step(self) -> None:
        """Execute a single step of the agent."""
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            tools=self._tools,
            tool_choice='auto',
            temperature=0.5
        )
        msg = completion.choices[0].message
        self._finish_reason = completion.choices[0].finish_reason
        self.messages.append(msg)


    def _execute_tool(self, tool_call_message: Any) -> None:
        """Execute a tool call."""
        tool_call_id = tool_call_message.id
        tool_name = tool_call_message.function.name
        tool_args = json.loads(tool_call_message.function.arguments)
        tool_result = execute_tool(tool_call_id, tool_name, debug=self.debug, **tool_args)
        self.messages.append(tool_result)


In [ ]:
agent = Agent(PTG_USER_ID, debug=True)

# Chatbot UI
while True:
    query = input()
    if query == 'quit':
        break

    # Log the user's message
    print(f"User: {query}")

    # Execute the agent inference loop
    start = time.perf_counter()
    response, events = agent(query)
    end = time.perf_counter()

    # Log the agents message and the time it took to respond
    print(f"Agent: {response} ({round(end - start, 2)} seconds)")

    # Print the events if they exist
    if events:
        for event in events:
            print(event)

## Conclusions

After setting up the agent, it is clear that the agent needs a more effective way of narrowing down the results, when it writes a query. One possible way to do this would be to setup a vector index. The vector index would be used by the agent to write a natural language query that would return all locations that satisfy the discrete filters and they would be sorted by relevance. To do this, we could embed the reviews of associated with each location. Then, we could create a vector store with each location. The vector store would have the business ID stored in the metadata, along with the city and category (both as indexes). This way, we can first query the vector store, and get a list of general venues that are somewhat inline with what the user is looking for. Once that query has resolved, we will then use the Yelp business ID's of each result, to apply a filter on the query on the graph database. This way, we will be querying over the space of all venues that met a certain similarity score with the user's query and relating that to the space of the user's posts. 